In [1]:
##???
%load_ext autoreload
%autoreload 2

In [ ]:
pip install categorical_embedder

In [2]:
#Set up packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import categorical_embedder as ce

from sklearn.preprocessing import StandardScaler

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [6]:
#Load data
df = pd.read_csv('../data/raw/beer_reviews.csv')
#Load saved processed data
#traindf = pd.read_csv('../data/processed/traindf.csv')
#target = pd.read_csv('../data/processed/target.csv')

In [ ]:
#EDA
#df.head()
#df.info()
#df.describe()
#df.shape()

In [ ]:
#Find out how many different labels we are predicting
#df.brewery_name.unique()

In [ ]:
#Visualisations

#boxplot = df.boxplot(column=['review_overall', 'review_aroma', 'review_appearance','review_palate','review_taste'])
#boxplot_abv = df.boxplot(column=['beer_abv'])

#df.hist(column=['review_overall', 'review_aroma', 'review_appearance','review_palate','review_taste'])

In [ ]:
#Prepare data  - removed inplace=True as it wasn't saving to csv
traindf = df.copy()
traindf = traindf.dropna()
traindf.reset_index(drop=True)





In [8]:
#Pre-processing: Scaling. (Can turn this into a pipeline - after standardscaler, some reviews turned negative
##cat_cols = ['brewery_name']
num_cols = ['review_overall', 'review_aroma', 'review_appearance','review_palate','review_taste']

#Instantiate scaler and one hot encoder
##ohe = OneHotEncoder(sparse=False,handle_unknown='ignore')
sc = StandardScaler()

#Fit and transform columns
traindf[num_cols] = sc.fit_transform(traindf[num_cols])

In [9]:
#Encode categorical features and target

# ce.get_embedding_info identifies the categorical variables, # of unique values and embedding size and returns a dictionary
embedding_info = ce.get_embedding_info(traindf)
embedding_info

# ce.get_label_encoded_data integer label encodes the categorical variables and prepares it to feed it to neural network. C
#Can do next step of training shallow neural network for embeddings
traindf_encoded,encoders = ce.get_label_encoded_data(traindf)

traindf_encoded.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,4886,1234817823,-3.239994,-2.511302,-2.198210,30121,65,-3.317561,-3.162309,34371,5.0,47986
1,10325,4886,1235915097,-1.148720,-1.792233,-1.384289,30121,51,-1.109519,-1.103587,32297,6.2,48213
2,10325,4886,1235916604,-1.148720,-1.792233,-1.384289,30121,59,-1.109519,-1.103587,5313,6.5,48215
3,10325,4886,1234725145,-1.148720,-1.073164,-0.570368,30121,61,-1.845533,-1.103587,34370,5.0,47969
4,1075,1360,1293735206,0.245463,1.084042,0.243553,22693,9,0.362510,0.955134,8745,7.7,64883


In [10]:
#Separate features and target


target = traindf_encoded['beer_style']
feature = traindf_encoded.drop(['brewery_id','review_time','review_profilename','beer_name','beer_abv','beer_beerid','beer_style'], axis=1)


#Alternative
#featuredf = pd.DataFrame(traindf_encoded)
#featuredf.head
#cols = [1,2,3,4,5,6]
#featuredf = traindf_encoded[traindf_encoded.columns[cols]]


#Save locally

target.to_csv('../data/processed/target.csv')

feature.to_csv('../data/processed/feature.csv')

<bound method NDFrame.head of          brewery_name  review_overall  review_aroma  review_appearance  \
0                4886       -3.239994     -2.511302          -2.198210   
1                4886       -1.148720     -1.792233          -1.384289   
2                4886       -1.148720     -1.792233          -1.384289   
3                4886       -1.148720     -1.073164          -0.570368   
4                1360        0.245463      1.084042           0.243553   
...               ...             ...           ...                ...   
1586609          4617        1.639646      0.364974          -0.570368   
1586610          4617        0.245463      1.803111          -2.198210   
1586611          4617        0.942555     -0.354095          -1.384289   
1586612          4617        0.245463      1.084042           1.057473   
1586613          4617        1.639646      1.084042           1.057473   

         review_palate  review_taste  
0            -3.317561     -3.162309  
1  

In [13]:

#from src.data.splitsets import split_sets_random (ModuleNotFoundError: No module named 'src.data')

from sklearn.model_selection import train_test_split

X_data, X_test, y_data, y_test = train_test_split (feature, target, test_size=0.2, random_state=8)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=8)


In [14]:
#Initial run - Save split datasets
np.save('../data/processed/X_train', X_train)
np.save('../data/processed/X_val',   X_val)
np.save('../data/processed/X_test',  X_test)
np.save('../data/processed/y_train', y_train)
np.save('../data/processed/y_val',   y_val)
np.save('../data/processed/y_test',  y_test)

#Subsequent runs - load saved datasets

#X_train = np.load('../data/processed/X_train.npy')
#X_val   = np.load('../data/processed/X_val.npy'  )
#X_test  = np.load('../data/processed/X_test.npy' )
#y_train = np.load('../data/processed/y_train.npy')
#y_val   = np.load('../data/processed/y_val.npy'  )
#y_test  = np.load('../data/processed/y_test.npy' )

In [15]:
#Define class

class PytorchRegression(nn.Module):
    def __init__(self, num_features):
        super(PytorchRegression, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 128)
        self.layer_out = nn.Linear(128, 1)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)))
        x = self.layer_out(x)
        return (x)

In [16]:
model = PytorchRegression(X_train.shape[1])

In [17]:
#Define functioni to check cuda

def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

In [18]:
#Model use device (L5.1)
device = get_device()
model.to(device)


PytorchRegression(
  (layer_1): Linear(in_features=6, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=1, bias=True)
)

In [19]:
print(model)

PytorchRegression(
  (layer_1): Linear(in_features=6, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=1, bias=True)
)


In [20]:
#Define

class PytorchDataset(Dataset):
    """
    Pytorch dataset
    ...

    Attributes
    ----------
    X_tensor : Pytorch tensor
        Features tensor
    y_tensor : Pytorch tensor
        Target tensor

    Methods
    -------
    __getitem__(index)
        Return features and target for a given index
    __len__
        Return the number of observations
    to_tensor(data)
        Convert Pandas Series to Pytorch tensor
    """
        
    def __init__(self, X, y):
        self.X_tensor = self.to_tensor(X)
        self.y_tensor = self.to_tensor(y)
    
    def __getitem__(self, index):
        return self.X_tensor[index], self.y_tensor[index]
        
    def __len__ (self):
        return len(self.X_tensor)
    
    def to_tensor(self, data):
        return torch.Tensor(np.array(data))

In [21]:
#Convert all sets to Pytorch dataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [22]:
#Train model

criterion = nn.MSELoss()

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [28]:
#Define train function

def train_regression(train_data, model, criterion, optimizer, batch_size, device, scheduler=None, collate_fn=None):
    """Train a Pytorch regresssion model

    Parameters
    ----------
    train_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    optimizer: torch.optim
        Optimizer
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    scheduler : torch.optim.lr_scheduler
        Pytorch Scheduler used for updating learning rate
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        RMSE Score
    """
    
    # Set model to training mode
    model.train()
    train_loss = 0

    # Create data loader
    data = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:
        
        # Reset gradients
        optimizer.zero_grad()
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Make predictions
        output = model(feature)
        
        # Calculate loss for given batch
        loss = criterion(output, target_class)
        
        # Calculate global loss
        train_loss += loss.item()
        
        # Calculate gradients
        loss.backward()
        
        # Update Weights
        optimizer.step()
        
    # Adjust the learning rate
    if scheduler:
        scheduler.step()

    return train_loss / len(train_data), np.sqrt(train_loss / len(train_data))

In [29]:
#Define test function

def test_regression(test_data, model, criterion, batch_size, device, collate_fn=None):
    """Calculate performance of a Pytorch regresssion model

    Parameters
    ----------
    test_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        RMSE Score
    """    
    
    # Set model to evaluation mode
    model.eval()
    test_loss = 0

    # Create data loader
    data = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Set no update to gradients
        with torch.no_grad():
            
            # Make predictions
            output = model(feature)
            
            # Calculate loss for given batch
            loss = criterion(output, target_class)
            
            # Calculate global loss
            test_loss += loss.item()
            
    return test_loss / len(test_data), np.sqrt(test_loss / len(test_data))


In [30]:
N_EPOCHS = 5
BATCH_SIZE = 32

In [31]:

for epoch in range(N_EPOCHS):
    train_loss, train_rmse = train_regression(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_rmse = test_regression(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\tLoss: {train_loss:.4f}\t|\tRMSE: {train_rmse:.1f}')
    print(f'\t(valid)\tLoss: {valid_loss:.4f}\t|\tRMSE: {valid_rmse:.1f}')

Epoch: 0
	(train)	Loss: 43.8559	|	RMSE: 6.6
	(valid)	Loss: 35.7507	|	RMSE: 6.0
Epoch: 1
	(train)	Loss: 35.3668	|	RMSE: 5.9
	(valid)	Loss: 34.6449	|	RMSE: 5.9
Epoch: 2
	(train)	Loss: 34.9896	|	RMSE: 5.9
	(valid)	Loss: 34.7687	|	RMSE: 5.9
Epoch: 3
	(train)	Loss: 34.7697	|	RMSE: 5.9
	(valid)	Loss: 34.3569	|	RMSE: 5.9
Epoch: 4
	(train)	Loss: 34.6168	|	RMSE: 5.9
	(valid)	Loss: 34.4431	|	RMSE: 5.9


In [32]:
#save model
torch.save(model, "../models/pytorch_reg_beer1.pt")

In [33]:
#Evaluate

test_loss, test_rmse = test_regression(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tRMSE: {test_rmse:.1f}')

	Loss: 34.4168	|	RMSE: 5.9


In [ ]:
#Spare

#df.loc[df['brewery_id'] == 27]

from joblib import dump
dump(reg,  '../models/linear_poly_2.joblib')